In [1]:
import re
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
print('data:', round(os.stat('html.txt').st_size*1e-6, 2), 'mb')

data: 319.05 mb


In [5]:
with open('html.txt', 'r') as data:
    data = [page+'/html>' for page in data.read().split('/html>')[:-1]]

In [7]:
def page_to_df(data):
    failed=0
    rows=[]
    soup = BeautifulSoup(data)
    for listing in soup.find_all("div", id=re.compile("^resultBlockWrapper")):
        try: # check if listing has price
            new_row = {'price': ' '.join(listing.find('div', {'class':'m-srp-card__price'}).text.split()),
                       'title': ' '.join(listing.find('span',{'class':'m-srp-card__title'}).text.split()),}
            try:new_row.update({coord.attrs['itemprop']: coord.attrs['content']
                               for coord in listing.find('div',{'itemprop':'geo'}).find_all('meta')})
            except:pass

            try:new_row.update({summary_card.find_all('div')[0].text:summary_card.find_all('div')[1].text.strip()
                               for summary_card in listing.find_all('div', {'class':'m-srp-card__summary__item'})})
            except:pass
            rows.append(new_row)
        except: failed+=1
            # skip if no price.
    df = pd.DataFrame(rows)
    return len(rows), failed, df

In [8]:
success = 0
failed = 0
df = pd.DataFrame()
for i, page in enumerate(data):
    page_to_df(data[i])
    page_success, page_failed, page_df = page_to_df(data[i])
    success+=page_success
    failed+=page_failed
    df = pd.concat([df, page_df], sort=False)
    if i%30==0: print(f'{i} pages added...')
print('COMPLETED.')

0 pages added...
30 pages added...
60 pages added...
90 pages added...
120 pages added...
150 pages added...
180 pages added...
210 pages added...
240 pages added...
270 pages added...
300 pages added...
330 pages added...
360 pages added...
390 pages added...
420 pages added...
COMPLETED.


In [9]:
print(f'{success} listings added.\n{failed} listings not added')

10678 listings added.
92 listings not added


In [10]:
print(len(df), '--->', len(df)-sum(df.duplicated()))
df.drop_duplicates(inplace=True)

10678 ---> 2406


In [11]:
df.to_csv('data.csv', index=False)